In [1]:
sys.path.append('../jax_healpix/')
# from SPHT_jax import *
from YLM_jax_log import *
from YLM_jax import *
from utils import *
from reshape_utils import *

In [2]:
from scipy.special import binom, jn, loggamma
from scipy.special import eval_jacobi as jacobi

In [3]:
shift=1.e-2
theta=jnp.linspace(0+shift,jnp.pi/2-shift,20)
beta=jnp.cos(theta)
beta_s = jnp.sin(jnp.arccos(beta))

In [4]:
log_beta=jnp.log(beta)

In [5]:
log_beta_s, _ = logsumexp(
        2 * log_beta,
        jnp.zeros_like(log_beta),
        -1 * jnp.ones_like(log_beta),
        jnp.ones_like(log_beta),
    )  # sin^2=1-cos^2 = 1-beta^2
log_beta_s *= 0.5  # sqrt

In [6]:
log_beta_s<-100

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

In [7]:
jnp.log(beta_s)

DeviceArray([-4.6050873e+00, -2.3914948e+00, -1.7580738e+00,
             -1.3778799e+00, -1.1081483e+00, -9.0133119e-01,
             -7.3566985e-01, -5.9938484e-01, -4.8538846e-01,
             -3.8909677e-01, -3.0738509e-01, -2.3804165e-01,
             -1.7945652e-01, -1.3043417e-01, -9.0077542e-02,
             -5.7712141e-02, -3.2835245e-02, -1.5082036e-02,
             -4.2030802e-03, -5.0009548e-05], dtype=float32)

In [8]:
l_max=500
l=np.arange(l_max)
# l=np.arange(900,l_max)
m1=0
m2=l*1.

In [9]:
spin_max=2

In [10]:
%time ylm=sYLM_recur(beta=beta,spins=(0,2),l_max=l_max-1)
%time ylm_log=sYLM_recur_log(log_beta=jnp.log(beta),spins=(0,2),l_max=l_max-1)

/home/deep/repos/SPHT/notebooks/../jax_healpix/YLM_jax.py:60: RuntimeWarning: divide by zero encountered in log
  log_Alm = -1 * logdiffexp(2 * np.log(l), 2 * np.log(m))
/home/deep/repos/SPHT/notebooks/../jax_healpix/utils.py:15: RuntimeWarning: divide by zero encountered in log
  log_diff = np.log(1 - np.exp(log_b - log_a))
/home/deep/repos/SPHT/notebooks/../jax_healpix/utils.py:15: RuntimeWarning: invalid value encountered in log
  log_diff = np.log(1 - np.exp(log_b - log_a))
/home/deep/repos/SPHT/notebooks/../jax_healpix/YLM_jax.py:70: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt((2 * l + 1) * (l**2 - m**2) / (2 * l - 1))


CPU times: user 7.05 s, sys: 4.12 s, total: 11.2 s
Wall time: 10.7 s


/home/deep/repos/SPHT/notebooks/../jax_healpix/YLM_jax_log.py:58: RuntimeWarning: divide by zero encountered in log
  log_Alm = -1 * logdiffexp(2 * np.log(l), 2 * np.log(m))
/home/deep/repos/SPHT/notebooks/../jax_healpix/utils.py:33: RuntimeWarning: invalid value encountered in subtract
  log_s = max_Arr + np.log1p(sign_s * np.exp(min_Arr - max_Arr))
/home/deep/repos/SPHT/notebooks/../jax_healpix/YLM_jax_log.py:105: RuntimeWarning: divide by zero encountered in log
  log_m = np.log(m)
/home/deep/repos/SPHT/notebooks/../jax_healpix/YLM_jax_log.py:94: RuntimeWarning: divide by zero encountered in log
  return 0.5 * (np.log(2 * l + 1) + np.log(l**2 - m**2) - np.log(2 * l - 1))
/home/deep/repos/SPHT/notebooks/../jax_healpix/YLM_jax_log.py:94: RuntimeWarning: invalid value encountered in log
  return 0.5 * (np.log(2 * l + 1) + np.log(l**2 - m**2) - np.log(2 * l - 1))
/home/deep/repos/SPHT/notebooks/../jax_healpix/utils.py:33: RuntimeWarning: divide by zero encountered in log1p
  log_s = max

CPU times: user 969 ms, sys: 44.9 ms, total: 1.01 s
Wall time: 1.05 s


In [11]:
jnp.log(beta)

DeviceArray([-5.0009548e-05, -4.2030802e-03, -1.5082036e-02,
             -3.2835308e-02, -5.7712205e-02, -9.0077609e-02,
             -1.3043417e-01, -1.7945658e-01, -2.3804188e-01,
             -3.0738509e-01, -3.8909677e-01, -4.8538876e-01,
             -5.9938496e-01, -7.3566985e-01, -9.0133131e-01,
             -1.1081486e+00, -1.3778805e+00, -1.7580748e+00,
             -2.3914969e+00, -4.6051922e+00], dtype=float32)

In [12]:
ylm2=ylm_log

In [13]:
lt=3
mt=1
ylm[0][lt,mt,:],ylm2[0][lt,mt,:]#,ylm_log[0][lt,mt,:]

(array([-0.0129265 , -0.11703702, -0.21455818, -0.30001712, -0.368657  ,
        -0.41671727, -0.44165534, -0.44229669, -0.41890281, -0.37315327,
        -0.3080438 , -0.22770567, -0.1371581 , -0.04200671,  0.05189247,
         0.13878251,  0.21335077,  0.27104698,  0.30835487,  0.32300245]),
 array([-0.01292666, -0.1170375 , -0.21455804, -0.30001702, -0.36865699,
        -0.41671725, -0.4416553 , -0.44229674, -0.41890285, -0.37315333,
        -0.3080438 , -0.22770568, -0.13715812, -0.04200671,  0.05189247,
         0.13878249,  0.21335078,  0.27104699,  0.30835487,  0.32300245]))

In [14]:
log_Alm = log_A_lm(3, 1)
log_Alm_prev = log_A_lm(3 - 1, 1)

In [15]:
log_Alm,log_Alm_prev

(0.7379532599047887, 0.8047189562170503)

In [16]:
xx=jnp.isclose(ylm[0],ylm2[0])
jnp.all(xx)

DeviceArray(False, dtype=bool)

In [17]:
jnp.where(~xx)

(DeviceArray([  2,   3,   4, ..., 499, 499, 499], dtype=int32),
 DeviceArray([  1,   1,   1, ..., 496, 497, 498], dtype=int32),
 DeviceArray([ 0,  0,  0, ..., 17, 17, 17], dtype=int32))

In [18]:
jnp.all(ylm[0][jnp.where(~xx)]==0)

DeviceArray(False, dtype=bool)

In [19]:
ylm2[0][jnp.where(~xx)]

array([-0.00772574, -0.01292666, -0.0189208 , ...,  0.04726337,
       -0.01593618,  0.00422121])

In [20]:
jnp.where(~xx)

(DeviceArray([  2,   3,   4, ..., 499, 499, 499], dtype=int32),
 DeviceArray([  1,   1,   1, ..., 496, 497, 498], dtype=int32),
 DeviceArray([ 0,  0,  0, ..., 17, 17, 17], dtype=int32))

In [21]:
jnp.unique(jnp.where(~xx)[2])

DeviceArray([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,
             15, 16, 17, 18, 19], dtype=int32)

In [22]:
s=2
xx=jnp.isclose(ylm[s][:,:,0],ylm2[s][:,:,0])
jnp.all(xx)

DeviceArray(False, dtype=bool)

In [23]:
s=-2
xx=jnp.isclose(ylm[s][:,:,0],ylm2[s][:,:,0])
jnp.all(xx)

DeviceArray(False, dtype=bool)

In [24]:
lt,mt=jnp.where(~xx)

In [25]:
ylm2[s][jnp.where(~xx)]

array([[-3.15409870e-03, -2.88559297e-02, -5.43661504e-02, ...,
        -3.10670517e-01, -3.14068735e-01, -3.15375796e-01],
       [-5.90033431e-03, -5.37580714e-02, -1.00187334e-01, ...,
        -1.00187121e-01, -5.37582128e-02, -5.90001109e-03],
       [ 4.57060836e-03,  4.16411703e-02,  7.76046155e-02, ...,
         7.76046101e-02,  4.16409327e-02,  4.57012895e-03],
       ...,
       [ 1.54641988e-03, -5.67106360e-01,  7.67198852e-02, ...,
         1.68418875e-02, -1.63525781e-02,  1.59180377e-02],
       [-3.35802812e-04,  2.54057510e-02,  2.17973231e-01, ...,
         1.92342452e-03, -3.44674648e-03,  4.94068671e-03],
       [ 6.68730519e-05,  6.28284854e-01, -1.62902614e-01, ...,
        -1.94520122e-02,  1.88863089e-02, -1.83689035e-02]])

In [26]:
ylm[s][jnp.where(~xx)]

array([[-3.15424572e-03, -2.88561357e-02, -5.43662376e-02, ...,
        -3.10670529e-01, -3.14068742e-01, -3.15375800e-01],
       [-5.90246057e-03, -5.37586719e-02, -1.00187276e-01, ...,
        -1.00187129e-01, -5.37582159e-02, -5.90001129e-03],
       [ 4.57055205e-03,  4.16410046e-02,  7.76046776e-02, ...,
         7.76046077e-02,  4.16409344e-02,  4.57012881e-03],
       ...,
       [ 1.54588645e-03, -5.67091861e-01,  7.67205073e-02, ...,
         1.68418804e-02, -1.63525707e-02,  1.59180358e-02],
       [-3.35695033e-04,  2.54049742e-02,  2.17974632e-01, ...,
         1.92340861e-03, -3.44678677e-03,  4.94068580e-03],
       [ 6.68566968e-05,  6.28267270e-01, -1.62902376e-01, ...,
        -1.94520101e-02,  1.88862970e-02, -1.83689015e-02]])

In [39]:
ylm[s][jnp.where(~xx)]/ylm2[s][jnp.where(~xx)]

array([[0.99406064, 1.00002139, 1.00000344, ..., 0.99999995, 0.99999999,
        0.99999993],
       [0.99486189, 1.0000148 , 1.00000312, ..., 1.00000004, 1.        ,
        1.00000003],
       [0.99998715, 0.99999616, 1.00000094, ..., 1.00000011, 0.99999997,
        0.99999994],
       ...,
       [0.99972035, 1.00003471, 1.00000744, ..., 0.99999687, 1.0000099 ,
        0.99999982],
       [0.9997253 , 0.9999262 , 1.00000756, ..., 0.99999957, 0.9999993 ,
        0.99999991],
       [0.99978138, 1.0000012 , 1.0000246 , ..., 0.99999634, 1.0000123 ,
        0.9999999 ]])

In [27]:
s=2
lt=2
mt=2
th_i=10

alm=alpha_lm(lt,mt)
alm_log=log_alpha_lm(lt,mt)

ylm[s][lt,mt,th_i],ylm[s][lt,mt,th_i]/(np.sqrt(2*lt+1)/np.sqrt(4*np.pi))

/home/deep/repos/SPHT/notebooks/../jax_healpix/YLM_jax_log.py:94: RuntimeWarning: divide by zero encountered in log
  return 0.5 * (np.log(2 * l + 1) + np.log(l**2 - m**2) - np.log(2 * l - 1))


(0.23011518853218346, 0.3648087233213425)

In [28]:
beta[th_i],theta[th_i]

(DeviceArray(0.6776687, dtype=float32), DeviceArray(0.82620865, dtype=float32))

In [29]:
def log_binom(x,y):
    return loggamma(x+1)-loggamma(y+1)-loggamma(x-y+1)
def wigner_d2(m1, m2, theta, l, l_use_bessel=1.0e3):
    """
    Function to compute wigner small-d matrices.
    This maybe less accurate than our main jax functions which use recursion.
    """
    l=l[:,None,None]
    m2=m2[None,:,None]
    m1=m1*np.ones_like(m2)
    theta=theta[None,None,:]
    k = np.amin([l - m1, l - m2, l + m1, l + m2], axis=0)
    a = np.absolute(m1 - m2)
    lamb = m2 - m1  # lambda
    x=m2<=m1
    lamb[x]=0
    b = 2 * l - 2 * k - a

#     d_mat = np.sqrt(
#         binom(2 * l - k, k + a)
#     )
    d_mat=0.5*log_binom(2 * l - k, k + a)
    
#     d_mat /= np.sqrt(binom(k + b, b))
    d_mat-=0.5*log_binom(k + b, b)
    x = k+b < b
    d_mat[x] = -np.inf
  
    beta_s=np.sin(theta / 2.0)
    beta=np.cos(theta / 2.0)
    
    d_mat=d_mat+a*np.log(np.absolute(beta_s))+b*np.log(np.absolute(beta))
    sign_dmat=np.sign(beta_s)**a
    sign_dmat=sign_dmat*(np.sign(beta)**b)
    
    d_mat=sign_dmat*np.exp(d_mat)
#     d_mat[x]=0
    x = d_mat == 0
    d_mat *= jacobi(k, a, b, np.cos(theta))  # l
    
    d_mat*=(-1)**lamb[0,:,0][None,:,None]
    
    d_mat[x] = 0

    if l_use_bessel is not None:
        x = l[:,0,0] >= l_use_bessel

        d_mat[x,:,:]=jn(m1-m2,l[x,:,:]*theta)
    return d_mat


In [30]:
s=2
lt=2
mt=2
th_i=10

alm=alpha_lm(lt,mt)

wd=wigner_d2(s, np.atleast_1d(mt), np.atleast_1d(theta[th_i]), np.atleast_1d(lt))
wd2=wigner_d2(-s, np.atleast_1d(mt), np.atleast_1d(theta[th_i]), np.atleast_1d(lt))

ylm_t=wd*np.sqrt(2*lt+1)/np.sqrt(4*np.pi)
ylm_t2=wd2*np.sqrt(2*lt+1)/np.sqrt(4*np.pi)

wd,wd2,wd+wd2,beta[th_i],theta[th_i]
#https://www.wolframalpha.com/input?i=WignerD%5B%7B2%2C2%2C-2%7D%2C0%2C0.82620859%2C0%5D%2BWignerD%5B%7B2%2C2%2C2%7D%2C0%2C0.82620859%2C0%5D

(array([[[0.70364309]]]),
 array([[[0.02597438]]]),
 array([[[0.72961746]]]),
 DeviceArray(0.6776687, dtype=float32),
 DeviceArray(0.82620865, dtype=float32))

In [31]:
ylm[s][lt,mt,th_i]+ylm[-s][lt,mt,th_i], ylm[s][lt,mt,th_i]-ylm[-s][lt,mt,th_i]

(0.016384183681506903, 0.44384619338286)

In [32]:
ylm_t,ylm_t2

(array([[[0.44384619]]]), array([[[0.0163842]]]))

In [33]:
ylm[s][lt,mt,th_i]/((ylm_t+ylm_t2)/2),((ylm_t+ylm_t2)/2)

(array([[[0.99999998]]]), array([[[0.23011519]]]))

In [34]:
ylm[0][lt,mt,th_i]*(2*(mt**2-lt)-(lt*(lt-1)))/beta_s[th_i]**2 #+ ylm[0][lt-1,mt,th_i]*(2*alm*beta[th_i]/beta_s[th_i]**2)

DeviceArray(0.77254844, dtype=float32)

In [35]:
ylm[s][lt,mt,th_i]

0.23011518853218346

In [36]:
ylm[-s][lt,mt,th_i]

-0.21373100485067656